# 用特定于分组的值填充缺失值

对于缺失数据的清理工作，有时你会用dropna将其滤除，而有时则可能会希望用一个固定值或由数据集本身所衍生出来的值去填充NA值。这时就得使用fillna这个工具了。在下面这个例子中，用平均值填充NA值：

In [14]:
import pandas as pd,numpy as np

In [25]:
s=pd.Series(np.random.randn(6))
s[::2]=np.nan
s

0         NaN
1    1.564442
2         NaN
3    0.964925
4         NaN
5   -0.676263
dtype: float64

In [26]:
s.fillna(s.mean())


0    0.617701
1    1.564442
2    0.617701
3    0.964925
4    0.617701
5   -0.676263
dtype: float64

假设你需要对不同的分组填充不同的值。可能你已经猜到了，只需将数据分组，并使用apply和一个能够对各数据块调用fillna的函数即可。下面是一些有关于美国几个州的示例数据，这些州又被分为东部和西部：

In [42]:
states=['Ohio','New York','Vermont','Fllrida','Oregon','Nevada','California','Idaho']

In [43]:
group_key=['East']*4+['West']*4

In [51]:
data=pd.Series(np.random.randn(8),index=states)
data

Ohio          0.630302
New York     -1.614147
Vermont      -2.576900
Fllrida      -1.723335
Oregon       -1.142562
Nevada        1.040029
California   -0.010826
Idaho        -0.345898
dtype: float64

In [53]:
data[['Vermont','Nevada','Idaho']]=np.nan
data

Ohio          0.630302
New York     -1.614147
Vermont            NaN
Fllrida      -1.723335
Oregon       -1.142562
Nevada             NaN
California   -0.010826
Idaho              NaN
dtype: float64

In [54]:
data.groupby(group_key).mean()

East   -0.902393
West   -0.576694
dtype: float64

我们可以用分组平均去填充NA值，如下：

In [55]:
fill_mean=lambda g:g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio          0.630302
New York     -1.614147
Vermont      -0.902393
Fllrida      -1.723335
Oregon       -1.142562
Nevada       -0.576694
California   -0.010826
Idaho        -0.576694
dtype: float64

此外，也可以在代码中预定义各组的填充值。由于分组具有一个name属性，所以我们可以拿来用一下：

In [56]:
fill_values={'East':0.5,'West':-1}
fill_func=lambda g:g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio          0.630302
New York     -1.614147
Vermont       0.500000
Fllrida      -1.723335
Oregon       -1.142562
Nevada       -1.000000
California   -0.010826
Idaho        -1.000000
dtype: float64

# 随机采样和排列

假设你要从一个大数据集中随机抽取样本以进行蒙特卡罗模拟（Monte Carlo simulation）或其他分析工作。‘抽取’的方式有很多，其中一些的效率会比其他的高很多。一个办法是，选取np.random.permutation(N)的前K个元素，其中N为完整数据的大小，K为期望的样本大小，作为一个更有趣的例子，下面是构造一副英语型扑克牌的一个方式：

In [60]:
#红桃（Hearts)，黑桃（Spades），梅花（Clubs），方片（Diamonds）

In [67]:
suits=['H','S','C','D']
card_val=(list(range(1,11))+[10]*3)*4
base_names=['A']+list(range(2,11))+['J','Q','K']
cards=[]
for suit in suits:
    cards.extend(str(num)+suit for num in base_names)
deck=pd.Series(card_val,index=cards)
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
QH     10
KH     10
dtype: int64

现在根据我们上面讲的，从整幅牌中抽出5张，代码如下：

In [70]:
def draw(deck,n=5):
    return deck.take(np.random.permutation(len(deck))[:n])#np.random.permutation(s),s可以是数组，可以是长度,将其打乱后返回
draw(deck)

KC     10
6S      6
3C      3
5C      5
10S    10
dtype: int64

<table><td bgcolor=red>pandas.DataFrame.take or pandas.Series.take
<table><td bgcolor=red>np.random.permutation

假设你想要从每种花色种随机抽取两张牌。由于花色是牌名的最后一个字符，所以我们可以据此进行分组，并使用apply:

In [75]:
get_suit=lambda card:card[-1]#只要最后一个字母就可以

In [77]:
deck.groupby(get_suit).apply(draw,n=2)

C  3C      3
   2C      2
D  10D    10
   QD     10
H  QH     10
   JH     10
S  3S      3
   AS      1
dtype: int64

另一种方法

In [104]:
deck.groupby(get_suit,group_keys=False).apply(draw,n=2)

AC     1
4C     4
QD    10
7D     7
9H     9
5H     5
9S     9
AS     1
dtype: int64

In [80]:
np.random.permutation(len(deck))

array([27,  1, 29, 38, 15,  2, 51, 30, 40, 49, 25, 10, 18, 48, 43, 20, 11,
        9, 21, 14, 47, 13,  0,  6, 19,  4, 46,  5, 31, 44, 26, 33, 16, 35,
        3, 17, 36, 50, 28, 39, 32, 12,  7, 23, 34, 37,  8, 45, 22, 41, 24,
       42])

In [114]:
df=pd.DataFrame({'a':np.random.randn(20),'b':[chr(np.random.randint(97,122))+chr(np.random.randint(97,122)) for i in range(20)]})

In [115]:
df

,a,b
0,0.050922,mm
1,2.568342,oh
2,-0.381910,wj
3,-0.424491,wu
4,0.379709,ij
5,0.543140,ny
6,1.508626,cy
7,0.639875,bl
8,-0.179666,xl
9,-1.065716,du


In [125]:
df.groupby(lambda x:x[-1],level=0).apply(draw,2)

,,a,b
0,0,0.050922,mm
1,1,2.568342,oh
2,2,-0.381910,wj
3,3,-0.424491,wu
4,4,0.379709,ij
5,5,0.543140,ny
6,6,1.508626,cy
7,7,0.639875,bl
8,8,-0.179666,xl
9,9,-1.065716,du
